In [1]:
import pandas as pd
import folium
import os

# 读取数据
df = pd.read_csv("./Failure_DB_List.csv")

# 假设列名是 'Custom location (Lat, Lon)'
df[['Latitude', 'Longitude']] = df['Custom location (Lat, Lon)'].str.split(',', expand=True)

# 转换为浮点数
df['Latitude'] = df['Latitude'].astype(float)
df['Longitude'] = df['Longitude'].astype(float)

country_coords = {
    'USA': [37.0902, -95.7129],
    'Japan': [36.2048, 138.2529],
    'UK': [55.3781, -3.4360],
    'South Korea': [35.9078, 127.7669],
    'Belgium': [50.5039, 4.4699],
    'China': [35.8617, 104.1954],
    'Germany': [51.1657, 10.4515],
    'Australia': [-25.2744, 133.7751]
}

# 2. 提取国家名（如果是像 "South Korea, YoungCheon City"）
df['Country Extracted'] = df['Location'].apply(lambda x: x.split(',')[0].strip() if pd.notna(x) else None)

# 3. 定义填充函数
def fill_missing_coords(row):
    if pd.isna(row['Latitude']) or pd.isna(row['Longitude']):
        country = row['Country Extracted']
        if country in country_coords:
            lat, lon = country_coords[country]
            return pd.Series({'Latitude': lat, 'Longitude': lon})
    return pd.Series({'Latitude': row['Latitude'], 'Longitude': row['Longitude']})

# 4. 应用填补
df[['Latitude', 'Longitude']] = df.apply(fill_missing_coords, axis=1)

In [3]:
m = folium.Map(location=[20, 0], zoom_start=2)

for _, row in df.iterrows():
    popup_text = f"""
    <b>Event Date:</b> {row['Event Date']}<br>
    <b>Capacity (MWh):</b> {row['Capacity (MWh)']}<br>
    <b>Battery Modules:</b> {row['Battery Modules']}<br>
    <b>Root Cause:</b> {row['Root Cause']}
    """
    folium.Marker(
        location=[row['Latitude'], row['Longitude']],
        popup=popup_text,
        icon=folium.Icon(color='red', icon='fire')
    ).add_to(m)

# 输出 HTML 文件到 templates 文件夹（供 Flask 使用）
os.makedirs("templates", exist_ok=True)

In [4]:
m.save("templates/storage_fire_map.html")